## Week 4: Amazon data
* Read Amazon.csv
* Get to know the data
* Create a smaller subset of the data

In [ ]:
# imports and specifications
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### read Amazon.csv

In [ ]:
amazon = pd.read_csv('/Users/Aaron_hill/Dropbox/data/Amazon.csv')
#amazon = pd.read_csv('/Users/aaron/Dropbox/data/Amazon.csv')

### get to know the data

In [ ]:
print("amazon is:", type(amazon))
print("amazon has", amazon.shape[0], "rows and", amazon.shape[1], "columns", "\n")
print("the data types for each of the columns in amazon:")
print(amazon.dtypes, "\n")
print("the first 10 rows in amazon:")
print(amazon.head(10))

### create a smaller subset of the data

In [ ]:
# create a subset of "amazon" that contains all the columns but only only the first 1000 rows
amazon_subset = amazon[:1000]
print(type(amazon_subset))
print(amazon_subset.shape)

### create a ndarray for `L`

In [ ]:
L = amazon_subset["helpful"]
print(type(L))
print(type(L.values))
print(L.shape)

### create a ndarray for `X`
Use only "Score" and "Time" as features, for now.

In [ ]:
X = amazon_subset[["Score", "Time"]]
print(type(X))
print(type(X.values))
print(X.shape)

### using the `SVC` class in `sklearn.svm`, fit the SVM model according to the given training data
For now, accept all the default parameters in `SVC`

In [ ]:
from sklearn.svm import SVC
clf = SVC() # accepting all the default parameters
clf.fit(X, L)

### how well did we do? compare the model's predictions for  `Y` to the labels `L`

In [ ]:
# number of accurate predictions
print(sum(clf.predict(X) == L.values))

### using the `SGDClassifier` class in `linear_model`, fit the model according to given training data

In [ ]:
from sklearn import linear_model
sgd = linear_model.SGDClassifier()
sgd.fit(X, L)

### how well did we do? compare the model's predictions for  `Y` to the labels `L`

In [ ]:
# number of accurate predictions
print(sum(sgd.predict(X) == L.values))